In [1]:
import requests
import json
from config.settings import Config
from src.database import SessionLocal, Job

# ============================================
# CONFIGURATION - CHANGE THIS
# ============================================
JOB_ID = 1  # <<< CHANGE THIS TO TEST DIFFERENT JOBS

print("="*80)
print(f"Testing job ID: {JOB_ID}")
print(f"Using LLM Model: {Config.llm_model}")
print(f"LLM API: {Config.llm_api}")
print("="*80)

# ============================================
# FETCH JOB FROM DATABASE
# ============================================
db = SessionLocal()
try:
    job = db.query(Job).filter(Job.id == JOB_ID).first()
    
    if not job:
        print(f"\n❌ Job with ID {JOB_ID} not found in database!")
        exit()
    
    print(f"\n✅ Job found:")
    print(f"   Title: {job.job_title}")
    print(f"   Company: {job.company_name}")
    print(f"   Site: {job.site}")
    print(f"   URL: {job.job_url}")
    print(f"   Description length: {len(job.job_description) if job.job_description else 0} chars")
    print(f"   Created: {job.created_at}")
finally:
    db.close()

# ============================================
# CHECK IF JOB HAS DESCRIPTION
# ============================================
if not job.job_description:
    print("\n⚠️  Job has no description to parse!")
    exit()

print("\n" + "="*80)
print("JOB DESCRIPTION (to be parsed):")
print("="*80)
desc = job.job_description
if len(desc) > 2000:
    print(desc[:2000] + f"\n\n... [truncated {len(desc) - 2000} more characters]")
else:
    print(desc)
print("="*80)

# ============================================
# PREPARE LLM REQUEST
# ============================================
user_message = f"""
Extract structured information from this job posting and return it as JSON matching this schema:

{Config.job_to_db_prompt}

Job Posting:
Title: {job.job_title}
Company: {job.company_name}
URL: {job.job_url}

Description:
{job.job_description[:Config.max_body_text_length]}

Return ONLY valid JSON, no additional text.
"""

headers = {
    "Authorization": f"Bearer {Config.llm_api_key}",
    "Content-Type": "application/json"
}

payload = {
    "model": Config.llm_model,
    "messages": [
        {
            "role": "system",
            "content": "You are a job posting parser. Extract structured information and return only valid JSON. IMPORTANT: All extracted text fields must be in English, regardless of the input language. Translate all content (job titles, skills, responsibilities, locations, etc.) to English for consistency."
        },
        {
            "role": "user",
            "content": user_message
        }
    ],
    "temperature": 0.1,
    "response_format": {"type": "json_object"}
}

print("\n" + "="*80)
print("SYSTEM PROMPT:")
print("="*80)
print(payload["messages"][0]["content"])

print("\n" + "="*80)
print("USER PROMPT:")
print("="*80)
print(user_message)

print("\n" + "="*80)
print("API REQUEST INFO:")
print("="*80)
print(f"URL: {Config.llm_api}/chat/completions")
print(f"Model: {Config.llm_model}")
print(f"Temperature: {payload['temperature']}")
print(f"Response Format: JSON")

# Estimate token count (rough estimation: 1 token ≈ 4 characters for English text)
system_prompt_chars = len(payload["messages"][0]["content"])
user_prompt_chars = len(payload["messages"][1]["content"])
total_chars = system_prompt_chars + user_prompt_chars
estimated_tokens = total_chars / 4

print(f"\n📊 Context Estimation:")
print(f"   System prompt: ~{system_prompt_chars:,} chars (~{int(system_prompt_chars/4):,} tokens)")
print(f"   User prompt: ~{user_prompt_chars:,} chars (~{int(user_prompt_chars/4):,} tokens)")
print(f"   Total input: ~{total_chars:,} chars (~{int(estimated_tokens):,} tokens)")
print("="*80)

# ============================================
# SEND REQUEST TO LLM
# ============================================
print("\n🔄 Sending request to LLM...\n")

try:
    response = requests.post(
        f"{Config.llm_api}/chat/completions",
        headers=headers,
        json=payload,
        timeout=60
    )
    
    print(f"Response Status: {response.status_code}")
    print("="*80)
    print("RAW API RESPONSE:")
    print("="*80)
    print(json.dumps(response.json(), indent=2))
    print("="*80)
    
    if response.status_code == 200:
        result = response.json()
        
        if "choices" in result and len(result["choices"]) > 0:
            content = result["choices"][0]["message"]["content"]
            
            print("\n" + "="*80)
            print("EXTRACTED JSON RESPONSE:")
            print("="*80)
            print(content)
            print("="*80)
            
            # Try to parse as JSON
            try:
                parsed_data = json.loads(content)
                print("\n✅ Successfully parsed JSON!")
                print("\n" + "="*80)
                print("PARSED DATA (Pretty Print):")
                print("="*80)
                print(json.dumps(parsed_data, indent=2))
                print("="*80)
                
                # Show some key fields
                print("\n📊 Key Extracted Fields:")
                print(f"   Job Function: {parsed_data.get('job_function', 'N/A')}")
                print(f"   Seniority: {parsed_data.get('seniority_level', 'N/A')}")
                print(f"   Industry: {parsed_data.get('industry', 'N/A')}")
                print(f"   Employment Type: {parsed_data.get('employment_type', 'N/A')}")
                print(f"   Remote Work: {parsed_data.get('remote_work', 'N/A')}")
                print(f"   Salary Range: {parsed_data.get('min_salary', 'N/A')} - {parsed_data.get('max_salary', 'N/A')} {parsed_data.get('salary_currency', '')}")
                print(f"   Hard Skills: {', '.join(parsed_data.get('hard_skills', []))[:100]}")
                print(f"   City: {parsed_data.get('city', 'N/A')}")
                print(f"   Country: {parsed_data.get('country', 'N/A')}")
                
            except json.JSONDecodeError as e:
                print(f"\n❌ Failed to parse JSON: {e}")
        else:
            print("\n❌ No choices in response")
    else:
        print(f"\n❌ API request failed with status {response.status_code}")
        print(f"Response: {response.text}")
        
except Exception as e:
    print(f"\n❌ Error during LLM request: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*80)
print("TEST COMPLETE")
print("="*80)

Testing job ID: 1
Using LLM Model: nvidia/nemotron-nano-12b-v2-vl:free
LLM API: https://openrouter.ai/api/v1

✅ Job found:
   Title: Operator(oare) linia de îmbuteliere
   Company: castel mimi
   Site: jobber.md
   URL: https://jobber.md/job/productie-anenii-noi-castel-mimi-operatoroare-linia-de-imbuteliere/9281/
   Description length: 1965 chars
   Created: 2025-10-25 10:39:15.171583

JOB DESCRIPTION (to be parsed):
Candidați
Oferte companii
Încarcă CV

ro
Despre noi
Job-uri
Categorii
Companii
Orașe
Programe
Română
Engleză
Rusă
ro
MDL
Logare

Jobber
/
Operator(oare) linia de îmbuteliere

Operator(oare) linia de îmbuteliere
Castel Mimi
Acum
:
21-10-2025
Aplică
Salvează
Ascunde
Distribuie
Detalii job
Salariu
Negociabil
Tipul job-ului
Full time
Experiență
Nu contează
Locație
<p>Anenii Noi, s. Bulboaca, str. Dacia 1</p>
Descriere job
Suntem în căutarea unor persoane atente la detalii, bine organizate și pline de energie.
Responsabilități
Manipularea și pregătirea sticlelor sau altor recip